## 1. Chargement des Bibliothèques

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model, callbacks
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2
import os
import keras.backend as K
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

# Reproductibilité
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

## 2. Paramètres et Configuration

In [ ]:
IMG_SIZE = (32, 32) 
BATCH_SIZE = 8
EPOCHS = 100
VAL_SPLIT = 0.2
TEST_SPLIT = 0.1
DATA_PATH = 'Data Science/Datasets/Dataset Livrable 2 - patches/patches/processed/'

## 3. Chargement et Préparation des Données

In [ ]:
def load_data(data_path):
    # Vérification hiérarchique renforcée
    noisy_path = os.path.join(data_path, 'noisy')
    clean_path = os.path.join(data_path, 'clean')
    
    for path in [noisy_path, clean_path]:
        if not os.path.isdir(path):
            raise ValueError(f"Dossier introuvable: {path}")
    
    # Collecte avec vérification d'extension
    valid_ext = ['.jpg', '.jpeg', '.png']
    
    noisy_files = sorted([f for f in os.listdir(noisy_path) 
                         if os.path.splitext(f)[1].lower() in valid_ext])
    clean_files = sorted([f for f in os.listdir(clean_path)
                         if os.path.splitext(f)[1].lower() in valid_ext])

    # Lecture avec progression et gestion d'erreur
    noisy_imgs, clean_imgs = [], []
    for idx, (nfile, cfile) in enumerate(zip(noisy_files, clean_files)):
        noisy_img = cv2.imread(os.path.join(noisy_path, nfile))
        noisy_img = cv2.cvtColor(noisy_img, cv2.COLOR_BGR2RGB)
        clean_img = cv2.imread(os.path.join(clean_path, cfile))
        clean_img = cv2.cvtColor(clean_img, cv2.COLOR_BGR2RGB)
        
        if noisy_img is None:
            print(f"ERREUR: Échec de lecture de {nfile} (taille attendue: {IMG_SIZE})")
            continue
        if clean_img is None:
            print(f"ERREUR: Échec de lecture de {cfile} (taille attendue: {IMG_SIZE})")
            continue
        
        # Vérification de la taille
        if noisy_img.shape != (*IMG_SIZE, 3):
            print(f"Avertissement: {nfile} a une taille {noisy_img.shape}, ignoré")
            continue
            
        noisy_imgs.append(noisy_img)
        clean_imgs.append(clean_img)
        
        if (idx+1) % 10 == 0:
            print(f"Traité {idx+1}/{len(noisy_files)} paires")

    print(f"\nSuccès: {len(noisy_imgs)} paires valides sur {len(noisy_files)}")
    return np.array(noisy_imgs), np.array(clean_imgs)

# Chargement avec vérification
try:
    X_noisy, X_clean = load_data(DATA_PATH)
except Exception as e:
    print(f"Erreur: {e}")
    raise

# Vérification des dimensions
assert X_noisy.shape == X_clean.shape, "Dimensions incohérentes entre X_noisy et X_clean!"

# Normalisation [0,1]
X_noisy = X_noisy.astype('float32') / 255.0
X_clean = X_clean.astype('float32') / 255.0

# Split adaptatif pour petits datasets
TOTAL_SIZE = len(X_noisy)
print(f"Total d'images: {TOTAL_SIZE}")
if TOTAL_SIZE < 100:
    # Stratégie pour datasets réduits
    TEST_SPLIT = max(1, int(0.1 * TOTAL_SIZE))
    VAL_SPLIT = max(1, int(0.2 * TOTAL_SIZE))
else:
    TEST_SPLIT = 0.1
    VAL_SPLIT = 0.2

# Split avec shuffle stratifié
X_train, X_temp, y_train, y_temp = train_test_split(
    X_noisy, X_clean, 
    test_size=(VAL_SPLIT + TEST_SPLIT), 
    random_state=SEED,
    shuffle=True
)
# Creation of validation and test sets
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=TEST_SPLIT/(VAL_SPLIT + TEST_SPLIT), 
    random_state=SEED
)

print(f"\nSplit Final:")
print(f"- Train: {len(X_train)}")
print(f"- Val: {len(X_val)}")
print(f"- Test: {len(X_test)}")

## 4. Définition du Modèle

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model, callbacks
import numpy as np

# --- Residual Block ---
def ResidualConvBlock(x, filters):
    shortcut = x
    x = layers.Conv2D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    # Adapter le shortcut si le nombre de canaux ne correspond pas
    if shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, 1, padding='same')(shortcut)
    x = layers.Add()([shortcut, x])
    x = layers.Activation('relu')(x)
    return x

# --- Attention Block ---
class SpatialAttention(layers.Layer):
    def __init__(self, ratio=8, **kwargs):
        super().__init__(**kwargs)
        self.ratio = ratio

    # Override build method to create layers
    def build(self, input_shape):
        channels = input_shape[-1]
        self.dense1 = layers.Dense(channels // self.ratio, activation='relu', use_bias=False)
        self.dense2 = layers.Dense(channels, activation='sigmoid', use_bias=False)
        self.conv = layers.Conv2D(1, 7, padding='same', activation='sigmoid')
        super().build(input_shape)

    # Override call method to define forward pass
    def call(self, x):
        channels = x.shape[-1]
        avg_pool = layers.GlobalAveragePooling2D()(x)
        avg_pool = layers.Reshape((1, 1, channels))(avg_pool)
        avg_pool = self.dense1(avg_pool)
        avg_pool = self.dense2(avg_pool)
        max_pool = tf.reduce_max(x, axis=-1, keepdims=True)
        avg_pool_spatial = tf.reduce_mean(x, axis=-1, keepdims=True)
        concat = tf.concat([max_pool, avg_pool_spatial], axis=-1)
        spatial = self.conv(concat)
        return x * avg_pool * spatial

# --- U-Net with Residuals and Attention ---
def build_denoising_unet(input_shape=(32,32,3)):
    inputs = layers.Input(input_shape)
    # Encoder
    e1 = layers.Conv2D(64, 3, padding='same', activation='relu')(inputs)
    e1 = ResidualConvBlock(e1, 64)
    p1 = layers.MaxPooling2D()(e1)
    e2 = layers.Conv2D(128, 3, padding='same', activation='relu')(p1)
    e2 = ResidualConvBlock(e2, 128)
    p2 = layers.MaxPooling2D()(e2)
    # Bottleneck
    b = layers.Conv2D(256, 3, padding='same', activation='relu')(p2)
    b = ResidualConvBlock(b, 256)
    b = SpatialAttention()(b)
    b = layers.Dropout(0.3)(b)
    # Decoder
    d1 = layers.Conv2DTranspose(128, 3, strides=2, padding='same', activation='relu')(b)
    d1 = layers.Concatenate()([d1, e2])
    d1 = ResidualConvBlock(d1, 128)
    d2 = layers.Conv2DTranspose(64, 3, strides=2, padding='same', activation='relu')(d1)
    d2 = layers.Concatenate()([d2, e1])
    d2 = ResidualConvBlock(d2, 64)
    outputs = layers.Conv2D(3, 1, activation='sigmoid')(d2)
    return Model(inputs, outputs)

# --- Hybrid Loss (MSE + SSIM + Perceptual) ---
vgg = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(32,32,3))
loss_model = Model(inputs=vgg.input, outputs=vgg.get_layer('block3_conv3').output)
loss_model.trainable = False

# --- Custom Loss Function ---
def hybrid_loss(y_true, y_pred):
    ssim_loss = 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1.0))
    mse_loss = tf.reduce_mean(tf.square(y_true - y_pred))
    y_true_features = loss_model(y_true)
    y_pred_features = loss_model(y_pred)
    perceptual_loss = tf.reduce_mean(tf.square(y_true_features - y_pred_features))
    return 0.5*ssim_loss + 0.4*mse_loss + 0.1*perceptual_loss

# --- Metrics ---
def PSNR(y_true, y_pred):
    return tf.image.psnr(y_true, y_pred, max_val=1.0)
def SSIM(y_true, y_pred):
    return tf.image.ssim(y_true, y_pred, max_val=1.0)


## 5. Compilation du Modèle

In [ ]:
# --- Model Compile ---
model = build_denoising_unet(input_shape=(32,32,3))
model.compile(optimizer='adam', loss=hybrid_loss, metrics=[PSNR, SSIM])

model.summary()



## 6. Entraînement du Modèle

In [ ]:
# Ajoute ce callback pour réduire le learning rate si la val_loss stagne
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=8, min_lr=1e-6, verbose=1)

# Modifie l’EarlyStopping
early_stop = callbacks.EarlyStopping(patience=10, restore_best_weights=True)

# Ajoute du data augmentation simple (optionnel)
data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=20,
    brightness_range=[0.8,1.2]
)

# Entraînement du modèle avec validation et callbacks
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=16,
    callbacks=[early_stop]
)

## 7. Sauvegarde du Modèle

In [ ]:
# Sauvegarde du modèle
model.save('Data Science/deepvision-main/deepvision-main/notebooks/livrable2/denoising_unet_model.h5')
print("✅ Modèle sauvegardé sous 'denoising_unet_model.h5'")

## 8. Chargement du Modèle

In [ ]:
# Chargement du modèle
from tensorflow.keras.models import load_model

# Définir les objets personnalisés pour le chargement du modèle
custom_objects = {
    'hybrid_loss': hybrid_loss,
    'PSNR': PSNR,
    'SSIM': SSIM,
    'SpatialAttention': SpatialAttention  # Ajout de la couche personnalisée
}

# Charger le modèle sauvegardé
model = load_model('Data Science/deepvision-main/deepvision-main/notebooks/livrable2/denoising_unet_model.h5', custom_objects=custom_objects)
print("✅ Modèle chargé depuis 'denoising_unet_model.h5'")

## 9. Évaluation du Modèle

In [ ]:
# --- Test ---
test_results = model.evaluate(X_test, y_test, batch_size=16)
print(f"Test Loss: {test_results[0]:.4f}, PSNR: {test_results[1]:.2f}, SSIM: {test_results[2]:.4f}")

# --- Visualisation sur quelques patches ---
import matplotlib.pyplot as plt
idx = np.random.choice(len(X_test), 5, replace=False)
for i in idx:
    noisy = X_test[i]
    clean = y_test[i]
    denoised = model.predict(np.expand_dims(noisy, 0))[0]
    plt.figure(figsize=(9,3))
    plt.subplot(1,3,1); plt.imshow(noisy); plt.title("Noisy"); plt.axis('off')
    plt.subplot(1,3,2); plt.imshow(denoised); plt.title("Denoised"); plt.axis('off')
    plt.subplot(1,3,3); plt.imshow(clean); plt.title("Clean"); plt.axis('off')
    plt.show()

In [ ]:
import numpy as np

# Fonction pour reconstruire une image à partir de patches
def reconstruct_from_patches(patches, positions, image_shape, patch_size=32):
    """
    Reconstruit une image à partir de ses patches et positions.
    """
    h, w, c = image_shape
    recon = np.zeros((h, w, c), dtype=np.float32)
    weight = np.zeros((h, w, c), dtype=np.float32)
    for patch, (y, x) in zip(patches, positions):
        recon[y:y+patch_size, x:x+patch_size] += patch
        weight[y:y+patch_size, x:x+patch_size] += 1
    weight[weight == 0] = 1
    return recon / weight


In [ ]:
# Fonction pour extraire des patches d'une image
def extract_patches_and_positions(image, patch_size=32, stride=32):
    """
    Découpe une image en patches et retourne aussi les positions (y, x).
    """
    h, w, c = image.shape
    patches = []
    positions = []
    for y in range(0, h - patch_size + 1, stride):
        for x in range(0, w - patch_size + 1, stride):
            patch = image[y:y+patch_size, x:x+patch_size, :]
            patches.append(patch)
            positions.append((y, x))
    return np.array(patches), positions


## 10. Denoising et Reconstruction de 10 Patches Aléatoires

Dans cette section, nous sélectionnons une image bruitée et la découpons en plusieurs patches. Nous utilisons le modèle pour débruiter ces patches, puis nous reconstruisons l'image progressivement en affichant 10 patches aléatoires.

### Étapes :
1. Charger une image bruitée et propre correspondante.
2. Découper l'image bruitée en patches de taille fixe.
3. Débruiter les patches avec le modèle.
4. Reconstruire l'image progressivement en affichant 10 patches aléatoires et leur contribution à la reconstruction.
5. Comparer visuellement l'image reconstruite avec l'image propre de référence.

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random

def eval_denoising_on_first_image(model, noisy_dir, clean_dir, patch_size=32, stride=16):

    # Charger seulement la première image
    noisy_files = sorted([f for f in os.listdir(noisy_dir) if f.lower().endswith(('.jpg', '.png'))])
    clean_files = sorted([f for f in os.listdir(clean_dir) if f.lower().endswith(('.jpg', '.png'))])
    
    if not noisy_files or not clean_files:
        print("Aucune image trouvée dans les répertoires.")
        return

    # Lire la première image
    nfile = noisy_files[0]
    cfile = clean_files[0]

    noisy_img = cv2.imread(os.path.join(noisy_dir, nfile))
    noisy_img = cv2.cvtColor(noisy_img, cv2.COLOR_BGR2RGB)
    clean_img = cv2.imread(os.path.join(clean_dir, cfile))
    clean_img = cv2.cvtColor(clean_img, cv2.COLOR_BGR2RGB)
    h, w, c = noisy_img.shape

    # Padding pour couverture totale
    pad_h = (np.ceil(h / patch_size) * patch_size).astype(int) - h
    pad_w = (np.ceil(w / patch_size) * patch_size).astype(int) - w
    noisy_pad = cv2.copyMakeBorder(noisy_img, 0, pad_h, 0, pad_w, cv2.BORDER_REFLECT)
    clean_pad = cv2.copyMakeBorder(clean_img, 0, pad_h, 0, pad_w, cv2.BORDER_REFLECT)

    noisy_norm = noisy_pad.astype(np.float32) / 255.0
    clean_norm = clean_pad.astype(np.float32) / 255.0

    # Découper l’image en patches
    patches = []
    positions = []
    for y in range(0, noisy_pad.shape[0] - patch_size + 1, stride):
        for x in range(0, noisy_pad.shape[1] - patch_size + 1, stride):
            patch = noisy_norm[y:y+patch_size, x:x+patch_size, :]
            patches.append(patch)
            positions.append((y, x))
    patches = np.array(patches)

    # Prédire avec le modèle
    denoised_patches = model.predict(patches, verbose=0)

    # Visualiser la reconstruction progressive avec 10 patches aléatoires
    sample_indices = random.sample(range(len(positions)), 10)
    recon_step = np.zeros_like(noisy_pad, dtype=np.float32)
    weight_step = np.zeros_like(noisy_pad, dtype=np.float32)

    for i, idx in enumerate(sample_indices):
        y, x = positions[idx]
        patch = denoised_patches[idx]
        recon_step[y:y+patch_size, x:x+patch_size] += patch
        weight_step[y:y+patch_size, x:x+patch_size] += 1
        recon_vis = recon_step / np.maximum(weight_step, 1e-8)

        # Affichage
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 3, 1)
        plt.imshow((patch * 255).astype(np.uint8))
        plt.title(f"Patch {i+1} ajouté\nPosition: ({y}, {x})")
        plt.axis("off")

        plt.subplot(1, 3, 2)
        plt.imshow((recon_vis * 255).astype(np.uint8))
        plt.title("Reconstruction\nPartielle")
        plt.axis("off")

        plt.subplot(1, 3, 3)
        plt.imshow(clean_pad)
        plt.title("Image Clean (référence)")
        plt.axis("off")

        plt.suptitle(f"Étape {i+1} sur 10")
        plt.tight_layout()
        plt.show()

# Exemple d'utilisation :
noisy_dir = "Data Science/Datasets/Dataset Livrable 2 - patches/processed/noisy"
clean_dir = "Data Science/Datasets/Dataset Livrable 2 - patches/processed/clean"
eval_denoising_on_first_image(model, noisy_dir, clean_dir, patch_size=32, stride=2)


## 11. Denoising de 10 Images

Nous évaluons les performances du modèle sur 10 images bruitées. Pour chaque image :
- Nous affichons l'image bruitée, l'image débruitée par le modèle et l'image propre de référence.
- Nous calculons les métriques PSNR (Peak Signal-to-Noise Ratio) et SSIM (Structural Similarity Index Measure) pour évaluer la qualité de la débruitage.

### Étapes :
1. Charger 10 images bruitées et propres correspondantes.
2. Débruiter chaque image avec le modèle.
3. Afficher les résultats pour chaque image :
   - Image bruitée.
   - Image débruitée.
   - Image propre (référence).
4. Calculer et afficher les métriques PSNR et SSIM pour chaque image.
5. Résumer les performances moyennes sur les 10 images.

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim


# Fonction pour évaluer le modèle sur un dossier d'images
def eval_denoising_on_folder(model, noisy_dir, clean_dir, patch_size=32, stride=16, max_images=10):
    noisy_files = sorted([f for f in os.listdir(noisy_dir) if f.lower().endswith(('.jpg', '.png'))])
    clean_files = sorted([f for f in os.listdir(clean_dir) if f.lower().endswith(('.jpg', '.png'))])
    noisy_files = noisy_files[:max_images]
    clean_files = clean_files[:max_images]
    psnr_list, ssim_list = [], []

    # Boucle sur les fichiers pour comparer les images noisy et clean
    for nfile, cfile in zip(noisy_files, clean_files):
        noisy_img = cv2.imread(os.path.join(noisy_dir, nfile))
        noisy_img = cv2.cvtColor(noisy_img, cv2.COLOR_BGR2RGB)
        clean_img = cv2.imread(os.path.join(clean_dir, cfile))
        clean_img = cv2.cvtColor(clean_img, cv2.COLOR_BGR2RGB)
        h, w, c = noisy_img.shape

        # Padding pour couvrir toute l'image
        pad_h = (np.ceil(h / patch_size) * patch_size).astype(int) - h
        pad_w = (np.ceil(w / patch_size) * patch_size).astype(int) - w
        noisy_pad = cv2.copyMakeBorder(noisy_img, 0, pad_h, 0, pad_w, cv2.BORDER_REFLECT)
        clean_pad = cv2.copyMakeBorder(clean_img, 0, pad_h, 0, pad_w, cv2.BORDER_REFLECT)
        noisy_norm = noisy_pad.astype(np.float32) / 255.0
        clean_norm = clean_pad.astype(np.float32) / 255.0

        # Découpage en patches avec overlap
        patches = []
        positions = []
        for y in range(0, noisy_pad.shape[0] - patch_size + 1, stride):
            for x in range(0, noisy_pad.shape[1] - patch_size + 1, stride):
                patch = noisy_norm[y:y+patch_size, x:x+patch_size, :]
                patches.append(patch)
                positions.append((y, x))
        patches = np.array(patches)

        # Prédiction des patches
        denoised_patches = model.predict(patches, verbose=0)

        # Reconstruction de l'image à partir des patches
        recon = np.zeros_like(noisy_pad, dtype=np.float32)
        weight = np.zeros_like(noisy_pad, dtype=np.float32)
        idx = 0
        for y, x in positions:
            recon[y:y+patch_size, x:x+patch_size] += denoised_patches[idx]
            weight[y:y+patch_size, x:x+patch_size] += 1
            idx += 1
        recon = recon / np.maximum(weight, 1e-8)
        recon = recon[:h, :w]
        clean_crop = clean_norm[:h, :w]

        # Calcul métriques
        psnr_val = psnr(clean_crop, recon, data_range=1.0)
        ssim_val = ssim(clean_crop, recon, channel_axis=-1, data_range=1.0)
        psnr_list.append(psnr_val)
        ssim_list.append(ssim_val)

        # Affichage
        plt.figure(figsize=(12,4))
        plt.subplot(1,3,1); plt.imshow(noisy_img); plt.title("Noisy"); plt.axis('off')
        plt.subplot(1,3,2); plt.imshow((recon*255).astype(np.uint8)); plt.title(f"Denoised\nPSNR={psnr_val:.2f} SSIM={ssim_val:.3f}"); plt.axis('off')
        plt.subplot(1,3,3); plt.imshow(clean_img); plt.title("Clean"); plt.axis('off')
        plt.suptitle(nfile)
        plt.show()

    print(f"Average PSNR: {np.mean(psnr_list):.2f} dB")
    print(f"Average SSIM: {np.mean(ssim_list):.3f}")

# Utilisation :
noisy_dir = "Data Science/Datasets/Dataset Livrable 2 - patches/processed/noisy"
clean_dir = "Data Science/Datasets/Dataset Livrable 2 - patches/processed/clean"

eval_denoising_on_folder(model, noisy_dir, clean_dir, patch_size=32, stride=2, max_images=10)

## 12. Denoising de 2 Images avec Différents Niveaux de Bruit

Dans cette section, nous testons la robustesse du modèle en ajoutant différents niveaux de bruit ("salt & pepper") à deux images propres. Nous évaluons la capacité du modèle à débruiter ces images.

### Étapes :
1. Sélectionner deux images propres.
2. Ajouter des niveaux croissants de bruit "salt & pepper" (par exemple, 0%, 10%, 20%, ..., 100%).
3. Débruiter les images bruitées avec le modèle.
4. Afficher les résultats pour chaque niveau de bruit :
   - Image bruitée.
   - Image débruitée.
   - Image propre (référence).
5. Calculer les métriques PSNR et SSIM pour chaque niveau de bruit.
6. Résumer les performances sous forme de graphiques :
   - PSNR moyen en fonction du niveau de bruit.
   - SSIM moyen en fonction du niveau de bruit.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim

# Fonction pour ajouter du bruit salt & pepper à une image normalisée [0,1]
def add_salt_pepper_noise(image, noise_level):
    """
    Ajoute du bruit salt & pepper à une image normalisée [0,1].
    noise_level: proportion de pixels bruités (ex: 0.1 = 10%)
    """
    noisy = image.copy()
    h, w, c = noisy.shape
    num_pixels = int(noise_level * h * w)
    # Salt
    coords = [np.random.randint(0, i - 1, num_pixels) for i in (h, w)]
    noisy[coords[0], coords[1], :] = 1
    # Pepper
    coords = [np.random.randint(0, i - 1, num_pixels) for i in (h, w)]
    noisy[coords[0], coords[1], :] = 0
    return noisy



# Paramètres
noise_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0]
large_noisy_dir = "Data Science/Datasets/Dataset Livrable 2 - patches/processed/noisy"
large_clean_dir = "Data Science/Datasets/Dataset Livrable 2 - patches/processed/clean"
large_files = sorted([f for f in os.listdir(large_clean_dir) if f.lower().endswith(('.jpg', '.png'))])[:2]  # Moins d'images pour mieux voir

metrics = {level: {'psnr': [], 'ssim': []} for level in noise_levels}

# Boucle sur les fichiers pour comparer les images noisy et clean
for idx, fname in enumerate(large_files):
    clean_img = cv2.imread(os.path.join(large_clean_dir, fname))
    clean_img = cv2.cvtColor(clean_img, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0
    h, w, c = clean_img.shape

    # On affiche une image par ligne, plus grande
    fig, axes = plt.subplots(nrows=len(noise_levels)+1, ncols=2, figsize=(10, 3*(len(noise_levels)+1)))
    fig.suptitle(f"Reconstruction par patch - {fname}", fontsize=18, y=1.01)

    # Ligne 0 : clean
    axes[0, 0].imshow(clean_img)
    axes[0, 0].set_title("Clean", fontsize=14)
    axes[0, 0].axis('off')
    axes[0, 1].axis('off')

    # Boucle sur les niveaux de bruit
    for j, noise_level in enumerate(noise_levels):
        # Ajouter du bruit salt & pepper
        noisy_img = add_salt_pepper_noise(clean_img, noise_level)
        # Reconstruction par patch
        patches, positions = extract_patches_and_positions(noisy_img, patch_size=32, stride=2)
        # Normalisation des patches
        denoised_patches = model.predict(patches, verbose=0)
        # Reconstruction de l'image à partir des patches
        recon_img = reconstruct_from_patches(denoised_patches, positions, clean_img.shape, patch_size=32)
        recon_img = np.clip(recon_img, 0, 1)
        # Calcul des métriques
        psnr_val = psnr(clean_img, recon_img, data_range=1.0)
        ssim_val = ssim(clean_img, recon_img, channel_axis=-1, data_range=1.0)
        metrics[noise_level]['psnr'].append(psnr_val)
        metrics[noise_level]['ssim'].append(ssim_val)

        # Colonne 0 : bruitée
        axes[j+1, 0].imshow(noisy_img)
        axes[j+1, 0].set_title(f"Noise {noise_level}", fontsize=13)
        axes[j+1, 0].axis('off')
        # Colonne 1 : débruitée
        axes[j+1, 1].imshow(recon_img)
        axes[j+1, 1].set_title(f"Denoised\nPSNR={psnr_val:.2f} SSIM={ssim_val:.3f}", fontsize=13)
        axes[j+1, 1].axis('off')

    plt.tight_layout()
    plt.show()

# Moyennes globales
avg_metrics = {level: {'psnr': np.mean(values['psnr']), 'ssim': np.mean(values['ssim'])} for level, values in metrics.items()}

plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(noise_levels, [avg_metrics[level]['psnr'] for level in noise_levels], 'o-')
plt.title('Average PSNR vs Salt & Pepper Noise Level')
plt.xlabel('Noise Level')
plt.ylabel('PSNR (dB)')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(noise_levels, [avg_metrics[level]['ssim'] for level in noise_levels], 'o-')
plt.title('Average SSIM vs Salt & Pepper Noise Level')
plt.xlabel('Noise Level')
plt.ylabel('SSIM')
plt.grid(True)

plt.tight_layout()
plt.show()

## 13. Denoising sur une Image Personnalisée

Dans cette section, nous testons le modèle sur une image personnalisée. L'objectif est de débruiter une image bruitée et de visualiser les résultats, y compris les patchs débruités et l'image reconstruite.

### Étapes :
1. Charger une image bruitée personnalisée.
2. Découper l'image en patches de taille fixe.
3. Débruiter les patches avec le modèle.
4. Afficher quelques patchs débruités pour visualiser les résultats intermédiaires.
5. Reconstruire l'image complète à partir des patches débruités.
6. Comparer visuellement l'image bruitée et l'image débruitée reconstruite.

### Résultat attendu :
- Une comparaison visuelle entre l'image bruitée (entrée) et l'image débruitée (sortie).
- Une visualisation des patchs débruités pour mieux comprendre le fonctionnement du modèle.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Fonction pour débruiter une image et visualiser les patchs
def denoise_and_visualize_image(model, image_path, patch_size=32, stride=32, n_show=6):
    """
    Débruite une image (déjà bruitée), affiche quelques patchs débruités et la reconstruction finale.
    Args:
        model: modèle keras entraîné pour le débruitage
        image_path: chemin de l'image bruitée à traiter
        patch_size: taille des patchs (par défaut 32)
        stride: stride pour le découpage (par défaut 32)
        n_show: nombre de patchs à afficher
    """

    # Chargement de l'image bruitée
    noisy_img = cv2.imread(image_path)
    noisy_img = cv2.cvtColor(noisy_img, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0

    # Découpage en patchs
    h, w, c = noisy_img.shape
    patches = []
    positions = []
    for y in range(0, h - patch_size + 1, stride):
        for x in range(0, w - patch_size + 1, stride):
            patch = noisy_img[y:y+patch_size, x:x+patch_size, :]
            patches.append(patch)
            positions.append((y, x))
    patches = np.array(patches)

    # Prédiction patch par patch
    denoised_patches = model.predict(patches, verbose=0)

    # Affichage de quelques patchs débruités
    n_show = min(n_show, len(patches))
    plt.figure(figsize=(12, 3))
    for i in range(n_show):
        plt.subplot(1, n_show, i+1)
        plt.imshow(np.clip(denoised_patches[i], 0, 1))
        plt.title(f"Denoised patch {i+1}")
        plt.axis('off')
    plt.suptitle("Exemples de patchs débruités")
    plt.tight_layout()
    plt.show()

    # Reconstruction de l'image débruitée
    recon = np.zeros_like(noisy_img, dtype=np.float32)
    weight = np.zeros_like(noisy_img, dtype=np.float32)
    for patch, (y, x) in zip(denoised_patches, positions):
        recon[y:y+patch_size, x:x+patch_size] += patch
        weight[y:y+patch_size, x:x+patch_size] += 1
    recon_img = recon / np.maximum(weight, 1e-8)
    recon_img = np.clip(recon_img, 0, 1)

    # Affichage final
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.imshow(noisy_img)
    plt.title("Image bruitée (entrée)")
    plt.axis('off')
    plt.subplot(1,2,2)
    plt.imshow(recon_img)
    plt.title("Image débruitée (sortie)")
    plt.axis('off')
    plt.suptitle("Comparaison Noisy / Denoised")
    plt.tight_layout()
    plt.show()


# Exemple d'utilisation
image_path = "Data Science/Datasets/Dataset Livrable 2 - patches/test/IMG20250415164601.jpg"  # <-- Remplace par ton image
denoise_and_visualize_image(model, image_path, patch_size=32, stride=16, n_show=6)
